#Forewards Kinematics for a 2R robot


In [8]:
import numpy as np
from matplotlib import pyplot as plt


# foreward kinematics
def position(phi1,phi2):
    l1 =2
    l2=1

    pos = np.zeros(2) 
    pos[1] = l1*np.sin(phi1) + l2*np.sin(phi1+phi2)
    pos[0] = l1*np.cos(phi1) + l2*np.cos(phi1+phi2)     
    return pos


print(position(1,2))    


[0.09061212 1.82406198]


invers Kinematics

In [11]:

# Newton Iteration
tol = 0.001
K =1000
k =0
def position_invers(phi1,phi2):
    l2 =1
    l1=2
    k=0
    J= np.zeros((2,2))
    f = np.zeros(2)
    x = np.zeros(2)
    while k<K:
        k = k+1
        J = np.array([[ -l1*np.sin(phi1)-l2*np.sin(phi1+phi2), -l2*np.sin(phi1+phi2)],[l1*np.cos(phi1)+l2*np.cos(phi1+phi2), l2*np.cos(phi1+phi2)]])
        f = np.array([l1*np.cos(phi1)+l2*np.cos(phi1+phi2),l1*np.sin(phi1)+l2*np.sin(phi1+phi2)])
        ATA = np.dot(J.T,J)
        ATB = np.dot(J.T,f) 
        x = np.linalg.solve(ATA,ATB)
    



    print(x)
    return x




    
       
position_invers(1,2)

# Jacobean
    
        


    




    

[ 0.09221753 -1.83406032]


array([ 0.09221753, -1.83406032])